# Product Bundling and Recommendation

## Product Bundling

![Example Product Bundle. Credits: Burger King](https://cxl.com/wp-content/uploads/2018/10/burger-king-bundle.png)

<p style = 'text-align: center;'>Image Credits: Burger King</p>

Product bundling refers to grouping products or services together for sale as one package, as illustrated in the image above.

## References

- [Apriori Algorithm in Machine Learning](https://www.javatpoint.com/apriori-algorithm-in-machine-learning)
- [Implement FP Growth Algorithm in Python](https://codinginfinite.com/implement-fp-growth-algorithm-in-python/)
- [Introduction to Apriori Algorithm in Python](https://intellipaat.com/blog/data-science-apriori-algorithm/)
- [Product Sales Analysis Using Python](https://medium.com/swlh/product-sales-analysis-using-python-863b29026957)